# 03 - Interactive Viz

## Deadline

Wednesday November 8th, 2017 at 11:59PM

## Important Notes

- Make sure you push on GitHub your Notebook with all the cells already evaluated
- Note that maps do not render in a standard Github environment : you should export them to HTML and link them in your notebook.
- Remember that `.csv` is not the only data format. Though they might require additional processing, some formats provide better encoding support.
- Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you plan to implement!
- Please write all your comments in English, and use meaningful variable names in your code

## Background

In this homework we will be exploring interactive visualization, which is a key ingredient of many successful data visualizations (especially when it comes to infographics).

Unemployment rates are major economic metrics and a matter of concern for governments around the world. Though its definition may seem straightforward at first glance (usually defined as the number of unemployed people divided by the active population), it can be tricky to define consistently. For example, one must define what exactly unemployed means : looking for a job ? Having declared their unemployment ? Currently without a job ? Should students or recent graduates be included ? We could also wonder what the active population is : everyone in an age category (e.g. `16-64`) ? Anyone interested by finding a job ? Though these questions may seem subtle, they can have a large impact on the interpretation of the results : `3%` unemployment doesn't mean much if we don't know who is included in this percentage. 

In this homework you will be dealing with two different datasets from the statistics offices of the European commission ([eurostat](http://ec.europa.eu/eurostat/data/database)) and the Swiss Confederation ([amstat](https://www.amstat.ch)). They provide a variety of datasets with plenty of information on many different statistics and demographics at their respective scales. Unfortunately, as is often the case is data analysis, these websites are not always straightforward to navigate. They may include a lot of obscure categories, not always be translated into your native language, have strange link structures, â€¦ Navigating this complexity is part of a data scientists' job : you will have to use a few tricks to get the right data for this homework.

For the visualization part, install [Folium](https://github.com/python-visualization/folium) (*HINT*: it is not available in your standard Anaconda environment, therefore search on the Web how to install it easily!). Folium's `README` comes with very clear examples, and links to their own iPython Notebooks -- make good use of this information. For your own convenience, in this same directory you can already find two `.topojson` files, containing the geo-coordinates of 

- European countries (*liberal definition of EU*) (`topojson/europe.topojson.json`, [source](https://github.com/leakyMirror/map-of-europe))
- Swiss cantons (`topojson/ch-cantons.topojson.json`) 

These will be used as an overlay on the Folium maps.

## Assignment

1. Go to the [eurostat](http://ec.europa.eu/eurostat/data/database) website and try to find a dataset that includes the european unemployment rates at a recent date.

   Use this data to build a [Choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) which shows the unemployment rate in Europe at a country level. Think about [the colors you use](https://carto.com/academy/courses/intermediate-design/choose-colors-1/), how you decided to [split the intervals into data classes](http://gisgeography.com/choropleth-maps-data-classification/) or which interactions you could add in order to make the visualization intuitive and expressive. Compare Switzerland's unemployment rate to that of the rest of Europe.

2. Go to the [amstat](https://www.amstat.ch) website to find a dataset that includes the unemployment rates in Switzerland at a recent date.

   > *HINT* Go to the `details` tab to find the raw data you need. If you do not speak French, German or Italian, think of using free translation services to navigate your way through. 

   Use this data to build another Choropleth map, this time showing the unemployment rate at the level of swiss cantons. Again, try to make the map as expressive as possible, and comment on the trends you observe.

   The Swiss Confederation defines the rates you have just plotted as the number of people looking for a job divided by the size of the active population (scaled by 100). This is surely a valid choice, but as we discussed one could argue for a different categorization.

   Copy the map you have just created, but this time don't count in your statistics people who already have a job and are looking for a new one. How do your observations change ? You can repeat this with different choices of categories to see how selecting different metrics can lead to different interpretations of the same data.

3. Use the [amstat](https://www.amstat.ch) website again to find a dataset that includes the unemployment rates in Switzerland at recent date, this time making a distinction between *Swiss* and *foreign* workers.

   The Economic Secretary (SECO) releases [a monthly report](https://www.seco.admin.ch/seco/fr/home/Arbeit/Arbeitslosenversicherung/arbeitslosenzahlen.html) on the state of the employment market. In the latest report (September 2017), it is noted that there is a discrepancy between the unemployment rates for *foreign* (`5.1%`) and *Swiss* (`2.2%`) workers. 

   Show the difference in unemployment rates between the two categories in each canton on a Choropleth map (*hint* The easy way is to show two separate maps, but can you think of something better ?). Where are the differences most visible ? Why do you think that is ?

   Now let's refine the analysis by adding the differences between age groups. As you may have guessed it is nearly impossible to plot so many variables on a map. Make a bar plot, which is a better suited visualization tool for this type of multivariate data.

4. *BONUS*: using the map you have just built, and the geographical information contained in it, could you give a *rough estimate* of the difference in unemployment rates between the areas divided by the [RÃ¶stigraben](https://en.wikipedia.org/wiki/R%C3%B6stigraben)?


In [1]:
#Import libraries
import os
import pandas as pd
import json
import folium
import numpy as np

# Task 1 : Europe unemployment

Beginning by reading the .tsv data into a DataFrame

In [2]:
urate_eur_month = r'une_rt_m.tsv'

data = pd.read_table(urate_eur_month)
data.head(5)

,"s_adj,age,unit,sex,geo\time",2017M09,2017M08,2017M07,2017M06,2017M05,2017M04,2017M03,2017M02,2017M01,...,1983M10,1983M09,1983M08,1983M07,1983M06,1983M05,1983M04,1983M03,1983M02,1983M01
0,"NSA,TOTAL,PC_ACT,F,AT",:,5.2,4.4,4.9,5.1,4.7,5.4,4.9,5.3,...,:,:,:,:,:,:,:,:,:,:
1,"NSA,TOTAL,PC_ACT,F,BE",:,7.7,7.6,6.8,6.9,7.4,8.0,8.2,7.8 b,...,:,:,:,:,:,:,:,:,:,:
2,"NSA,TOTAL,PC_ACT,F,BG",:,5.3,5.3,5.3,5.6,6.2,6.8,7.1,7.0,...,:,:,:,:,:,:,:,:,:,:
3,"NSA,TOTAL,PC_ACT,F,CY",:,12.2,12.4,11.4,10.7,11.4,13.1,13.9,14.0,...,:,:,:,:,:,:,:,:,:,:
4,"NSA,TOTAL,PC_ACT,F,CZ",:,3.5,3.5,3.5,3.5,4.2,4.4,4.3,4.2,...,:,:,:,:,:,:,:,:,:,:


For the composition of our map, we only choose a fraction of the unemployment rate data: the country names with their respective unemployment values for the month of June 2017. 
The data is seasonally averaged, exprimed in percentage of the active population (float).

In [19]:
interesting_data = data[data['s_adj,age,unit,sex,geo\\time'].str.startswith('SA,TOTAL,PC_ACT,T')]
interesting_data = interesting_data[['s_adj,age,unit,sex,geo\\time','2017M06 ']]
interesting_data.columns = ['country','rate_2017_06']
interesting_data['rate_2017_06']= interesting_data['rate_2017_06'].astype(float)

Cleaning the names :

In [20]:
def clean_country(cname):
    cname = cname.replace('SA,TOTAL,PC_ACT,T,','')
    return cname    

In [21]:
interesting_data['country'] = interesting_data['country'].map(clean_country)

In [22]:
interesting_data = interesting_data.reset_index(drop=True)[['country','rate_2017_06']]

In [23]:
#interesting_data.to_csv('EU_countries_2017_06.csv',encoding='UTF-8',index=False)

Now we filter the country data, eliminating the useless countries :

In [24]:
filter_eur= interesting_data['country'].str.startswith(('EU','EA','US','JP')).map(lambda x : not x)
interesting_data = interesting_data[filter_eur]
interesting_data.head()

,country,rate_2017_06
0,AT,5.3
1,BE,7.2
2,BG,6.2
3,CY,10.9
4,CZ,2.9


Before starting with the Europe map, we will just render the full names of the countries on our Dataframe. To proceed, we are helped by the glossary of the country codes on eurostat website : http://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes

In [25]:

#We create a map from the ISO2 to the full country names 
full_countries_mapper = {geo_json_europe['features'][i]['properties']['ISO2'] : geo_json_europe['features'][i]['properties']['NAME'] for i in range(len(geo_json_europe['features']))}


#Handeling special cases
full_countries_mapper["EL"] = "Greece"
full_countries_mapper["UK"] = "United Kingdom"

interesting_data['country'] = interesting_data['country'].map(lambda c : full_countries_mapper[c])
interesting_data

,country,rate_2017_06
0,Austria,5.3
1,Belgium,7.2
2,Bulgaria,6.2
3,Cyprus,10.9
4,Czech Republic,2.9
5,Germany,3.8
6,Denmark,5.7
10,Estonia,6.5
11,Greece,21.3
12,Spain,17.1


We choosed the city of Radom, Poland, as the center of our Europe map.

In [26]:
radom = [51.4172,21.1562]
europe= folium.Map(radom, tiles='cartodbpositron', zoom_start=4)

Now that we have the empty map, we are going to add the Europe GeoJson paths.

In [27]:
europe_countries_path = r'./geojson/europe.geojson.json'
geo_json_europe = json.load(open(europe_countries_path))
#enlever les pays dont on n'a pas de data?
#topo_json_europe

We can now define a Choropleth representation on the Europe map with the unemployment data obtained before.

In [28]:
#folium.GeoJson(geo_json_europe,name='topojson').add_to(europe)

europe.choropleth(geo_data=geo_json_europe, data=interesting_data,
                  columns=['country','rate_2017_06'],
                  key_on='feature.properties.NAME',
                  fill_color='OrRd',
                  fill_opacity=0.7,
                  line_opacity=0.2,
                  legend_name='Percentage of active population that is unemployed (%)',
                  #threshold_scale = np.arange(interesting_data['rate_2017_06'].min(),interesting_data['rate_2017_06'].max(),1)
                 )
europe




The country from which we couldn't retrieve the data are in the lightest color of orange and are the following ones:

All the countries in the lightest color of orange are the ones for which we don't have any data.

# Task 2 : Swiss unemployment

In [29]:
swiss_un = pd.read_excel('un_rate_ch.xlsx')
swiss_un.iloc[1]

2.1 Taux de chômage            Canton
Unnamed: 1                       Mois
Unnamed: 2               Janvier 2017
Unnamed: 3               Janvier 2017
Unnamed: 4               Février 2017
Unnamed: 5               Février 2017
Unnamed: 6                  Mars 2017
Unnamed: 7                  Mars 2017
Unnamed: 8                 Avril 2017
Unnamed: 9                 Avril 2017
Unnamed: 10                  Mai 2017
Unnamed: 11                  Mai 2017
Unnamed: 12                 Juin 2017
Unnamed: 13                 Juin 2017
Unnamed: 14              Juillet 2017
Unnamed: 15              Juillet 2017
Unnamed: 16                 Août 2017
Unnamed: 17                 Août 2017
Unnamed: 18            Septembre 2017
Unnamed: 19            Septembre 2017
Unnamed: 20              Octobre 2017
Unnamed: 21              Octobre 2017
Unnamed: 22                     Total
Unnamed: 23                     Total
Name: 1, dtype: object

In [30]:
cantons_map = {
    'Zurich' : 'ZH',
    'Berne' : 'BE',
    'Lucerne' : 'LU',
    'Uri' : 'UR',
    'Schwyz' : 'SZ',
    'Obwald' : 'OW',
    'Nidwald' : 'NW',
    'Glaris' : 'GL',
    'Zoug' : 'ZG',
    'Fribourg' : 'FR',
    'Soleure' : 'SO',
    'Bâle-Ville' : 'BS',
    'Bâle-Campagne' : 'BL',
    'Schaffhouse' : 'SH',
    'Appenzell Rhodes-Extérieures' : 'AR',
    'Appenzell Rhodes-Intérieures' : 'AI',
    'St-Gall' : 'SG',
    'Grisons' : 'GR',
    'Argovie' : 'AG',
    'Thurgovie' : 'TG',
    'Tessin' : 'TI',
    'Vaud' : 'VD',
    'Valais': 'VS',
    'Neuchâtel' : 'NE',
    'Genève' : 'GE',
    'Jura' : 'JU'
}

In [31]:
#swiss_un = swiss_un
swiss_un = swiss_un.iloc[:,[0,18]]
swiss_un = swiss_un.drop([0,1,2,29])
swiss_un = swiss_un.reset_index(drop=True)
swiss_un.columns = ['Canton','Unemployment rate']
swiss_un['Unemployment rate'] = swiss_un['Unemployment rate'].astype(float)
swiss_un['ID'] = swiss_un['Canton'].map(lambda c : cantons_map[c])
swiss_un

,Canton,Unemployment rate,ID
0,Zurich,3.3,ZH
1,Berne,2.4,BE
2,Lucerne,1.7,LU
3,Uri,0.6,UR
4,Schwyz,1.7,SZ
5,Obwald,0.7,OW
6,Nidwald,1.0,NW
7,Glaris,1.8,GL
8,Zoug,2.3,ZG
9,Fribourg,2.7,FR


In [32]:
swiss_cantons_path = r'./topojson/ch-cantons.topojson.json'
swiss_cantons_topo = json.load(open(swiss_cantons_path))

In [33]:
ch = [46.8, 8.3]
switzerland = folium.Map(ch, tiles='cartodbpositron', zoom_start=8)

In [34]:
#folium.TopoJson(swiss_cantons_topo,'objects.cantons',name='topojson').add_to(switzerland)

In [35]:
switzerland.choropleth(
    geo_data=swiss_cantons_topo,
    data=swiss_un,
    columns=['ID','Unemployment rate'],
    key_on='id',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Percentage of active population that is unemployed (%)',
    topojson='objects.cantons'
)

In [36]:
switzerland

In [ ]:
#to save
#map_osm.save('/tmp/map.html')